<a href="https://colab.research.google.com/github/vishnubob/AoC2021/blob/main/AoC_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import itertools
import requests
import numpy as np
import scipy.signal

cache_dir = "/content/drive/MyDrive/AoC2021/"

session_fn = f"{cache_dir}/session_id"
with open(session_fn) as fh:
    session_id = fh.read().strip()

def aoc_input(day, split=True):
    url = f"https://adventofcode.com/2021/day/{day}/input"
    cache_fn = f"{cache_dir}/d{day}.dat"
    if not os.path.exists(cache_fn):
        cookies = {'session': session_id}
        resp = requests.get(url, cookies=cookies)
        data = resp.content.decode()
        with open(cache_fn, 'w') as fh:
            fh.write(data)
    with open(cache_fn) as fh:
        data = fh.read()
    data = data.strip()
    if split:
        data = data.split('\n')
    return data

In [ ]:
inp = aoc_input(1)

# day 1, puzzle 1
inp = np.array([int(x) for x in inp])
cnt = np.sum((inp[1:] - inp[:-1]) > 0)
print(f"Day 1, puzzle 1: {cnt}")

# day 1, puzzle 2
strides = (inp.strides[0], inp.strides[0])
shape = (len(inp) - 2, 3)
inp2 = np.lib.stride_tricks.as_strided(inp, shape=shape, strides=strides)
inp2 = np.sum(inp2, axis=1)
cnt = np.sum((inp2[1:] - inp2[:-1]) > 0)
print(f"Day 1, puzzle 2: {cnt}")

Day 1, puzzle 1: 1316
Day 1, puzzle 2: 1344


In [ ]:
inp = aoc_input(2)

# day 2, puzzle 1
axis_map = ["forward", "down", "up"]
axis_id = np.eye(3, dtype=np.int)
inp = [line.split(' ') for line in inp]
cols = np.array([int(val) * axis_id[axis_map.index(dr)] for (dr, val) in inp])
sums = np.sum(cols, axis=0)
prod = sums[0] * (sums[1] - sums[2])
print(f"Day 2, puzzle 1: {prod}")

# day 2, puzzle 2
aim = np.cumsum((cols[:, 1] + -cols[:, 2]))
prod = np.sum(cols[:, 0]) * np.sum(cols[:, 0] * aim)
print(f"Day 2, puzzle 2: {prod}")


Day 2, puzzle 1: 1499229
Day 2, puzzle 2: 1340836560


In [ ]:
inp = aoc_input(3)

# day 3, puzzle 1
n_items = len(inp)
shape = (n_items, -1)
inp = str.join('', inp).encode()
inp = (np.frombuffer(inp, dtype=np.uint8).reshape(shape) - ord('0')).astype(np.int)
bits = (2 ** np.arange(inp.shape[1]))[::-1]
gamma = np.sum(bits * (np.sum(inp.T, axis=1) > (inp.shape[0] // 2)))
prod = gamma * (~gamma & (2 ** inp.shape[1] - 1))
print(f"Day 3, puzzle 1: {prod}")

# day 3, puzzle 2
t_col = lambda tbl, col: (np.sum(tbl.T, axis=1) >= ((tbl.shape[0] + 1) // 2))[col]
t_mask = lambda tbl, col: tbl[tbl.T[col, :] == t_col(tbl, col)]
t_find = lambda tbl, col: tbl if tbl.shape[0] == 1 else t_find(t_mask(tbl, col), col + 1)
o2 = np.sum(bits * t_find(inp, 0))
t_col = lambda tbl, col: (np.sum(tbl.T, axis=1) <= ((tbl.shape[0] - 1) // 2))[col]
co2 = np.sum(bits * t_find(inp, 0))
prod = o2 * co2
print(f"Day 3, puzzle 2: {prod}")

Day 3, puzzle 1: 2648450
Day 3, puzzle 2: 2845944


In [ ]:
inp = aoc_input(4, split=False)

# day 4, puzzle 1

n_items = len(inp)
inp = inp.strip().split('\n\n')
n_boards = len(inp) - 1
calls = np.fromiter(map(int, inp[0].split(',')), dtype=np.int)
inp = map(int, re.sub('\s+', ' ', str.join(' ', inp[1:]).replace('\n', ' ')).split(' '))
inp = np.fromiter(inp, dtype=np.int)
dim = int((len(inp) / n_boards) ** 0.5)
inp = inp.reshape((n_boards, dim, dim))

def mark_and_check(call, inp, marks):
    bingo = -inp.shape[-1]
    marks[inp == call] = -1
    check = \
        (np.sum(marks.T, axis=0) == bingo) \
      | (np.sum(marks.T, axis=1) == bingo)
    if np.any(check):
        idx = np.argmax(check)
        winner = np.unravel_index(idx, check.shape)[1]
        idx = marks[winner] == 0
        return (winner, np.sum(inp[winner][idx]) * call)

def run_game(calls, inp, marks):
    res = mark_and_check(calls[0], inp, marks)
    if res is None:
        return run_game(calls[1:], inp, marks)
    return res[-1]

marks = np.zeros_like(inp)
prod = run_game(calls, inp, marks)
print(f"Day 4, puzzle 1: {prod}")

# day 4, puzzle 2

def find_last(calls, inp, marks, last_prod=None):
    if len(calls) == 0:
        return last_prod
    res = mark_and_check(calls[0], inp, marks)
    if res is not None:
        (winner, last_prod) = res
        inp = np.delete(inp, winner, 0)
        marks = np.delete(marks, winner, 0)
        return find_last(calls, inp, marks, last_prod)
    else:
        return find_last(calls[1:], inp, marks, last_prod)

marks = np.zeros_like(inp)
prod = find_last(calls, inp, marks)
print(f"Day 4, puzzle 2: {prod}")

Day 4, puzzle 1: 49860
Day 4, puzzle 2: 24628


In [ ]:
inp = aoc_input(5)

def line(pts, hv_only=True):
    ((x0, y0), (x1, y1)) = pts
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    xi = min(x0, x1)
    yi = min(y0, y1)
    vals = list()
    if dx == 0:
        vals = [[x0, y + yi] for y in range(dy + 1)]
    elif dy == 0:
        vals = [[x + xi, y0] for x in range(dx + 1)]
    elif not hv_only:
        xx = np.arange(0, dx + 1) + xi
        yy = np.arange(0, dy + 1) + yi
        if xx[0] != x0: xx = xx[::-1]
        if yy[0] != y0: yy = yy[::-1]
        vals = list(zip(xx, yy))
    return vals

line_re = re.compile("(\d+,\d+) -> (\d+,\d+)")
evk = lambda it: tuple(map(eval, line_re.match(it).groups()))
inp = [evk(it) for it in inp]

# day 5, puzzle 1

pts = itertools.chain(*[line(pts) for pts in inp])
pts = np.array(list(pts))
dim = np.max(pts.flatten()) + 1
cnts = np.zeros((dim, dim), dtype=np.int)
np.add.at(cnts, tuple(pts.T), 1)
max_cnt = np.sum(cnts > 1)
print(f"Day 5, puzzle 1: {max_cnt}")

# day 5, puzzle 2

pts = itertools.chain(*[line(pts, False) for pts in inp])
pts = np.array(list(pts))
dim = np.max(pts.flatten()) + 1
cnts = np.zeros((dim, dim), dtype=np.int)
np.add.at(cnts, tuple(pts.T), 1)
max_cnt = np.sum(cnts > 1)
print(f"Day 5, puzzle 2: {max_cnt}")

Day 5, puzzle 1: 4421
Day 5, puzzle 2: 18674


In [ ]:
inp = aoc_input(6, split=False)
inp = list(map(int, inp.split(',')))

def run_sim(counts, ttl=80):
    if ttl == 0:
        return counts
    parents = counts[:7]
    kids = counts[7:]
    parents = np.roll(parents, -1)
    new_kids = parents[-1]
    parents[-1] += kids[0]
    counts = np.concatenate((parents, kids[1:], [new_kids]))
    return run_sim(counts, ttl=ttl - 1)

# day 6, puzzle 1

counts = np.zeros((9,), dtype=np.int)
np.add.at(counts, inp, 1)
total = np.sum(run_sim(counts))
print(f"Day 6, puzzle 1: {total}")

# day 6, puzzle 2

counts = np.zeros((9,), dtype=np.int)
np.add.at(counts, inp, 1)
total = np.sum(run_sim(counts, ttl=256))
print(f"Day 6, puzzle 2: {total}")

Day 6, puzzle 1: 376194
Day 6, puzzle 2: 1693022481538


In [ ]:
inp = aoc_input(7, split=False)
inp = np.array(list(map(int, inp.split(',')))).astype(np.int)

# day 7, puzzle 1

delta = np.abs(inp - np.median(inp)).astype(np.int)
sum = np.sum(delta)
print(f"Day 7, puzzle 1: {sum}")

# day 7, puzzle 2

pos = int(np.floor(np.mean(inp)))
delta = np.abs(inp - pos).astype(np.int)
sum = np.sum([np.sum(np.arange(1, d + 1)) for d in delta])
print(f"Day 7, puzzle 2: {sum}")

Day 7, puzzle 1: 328262
Day 7, puzzle 2: 90040997


In [ ]:
inp = aoc_input(8)

segmap = [
  'abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg'
]

uniqs = [1, 4, 7, 8]
uniqs_len = [len(segmap[val]) for val in uniqs]

# day 8, puzzle 1

sigs = [[ex.split(' ') for ex in it.split(' | ')] for it in inp]
outp = list(zip(*sigs))[-1]
sums = sum([sum([1 if len(wd) in uniqs_len else 0 for wd in out]) for out in outp])

print(f"Day 8, puzzle 1: {sums}")

# day 8, puzzle 2

def fingerprint(sigs):
    segset = [set(it) for it in sigs]
    sigmap = np.zeros((len(segset), len(segset)))
    for (idx_1, it_1) in enumerate(segset):
        for (idx_2, it_2) in enumerate(segset):
            sigmap[idx_1][idx_2] = len(it_1 & it_2)
    sigmap = list(np.sum(sigmap, axis=0).astype(np.int))
    return sigmap


def eval_digits(sigs):
    for (inp_sigs, outp_sigs) in sigs:
        inp_fp = fingerprint(inp_sigs)
        val_map = {tuple(sorted(sig)): truth_fp.index(fp) for (sig, fp) in zip(inp_sigs, inp_fp)}
        vals = [val_map[tuple(sorted(sig))] for sig in outp_sigs]
        val = int(str.join('', map(str, vals)))
        yield val

truth_fp = fingerprint(segmap)
sums = sum(eval_digits(sigs))
print(f"Day 8, puzzle 2: {sums}")

Day 8, puzzle 1: 288
Day 8, puzzle 2: 940724


In [ ]:
inp = aoc_input(9)
n_items = len(inp)
shape = (n_items, -1)
inp = str.join('', inp).encode()
inp = (np.frombuffer(inp, dtype=np.uint8).reshape(shape) - ord('0')).astype(np.int)

# day 9, puzzle 1

conv_f = np.array([
  [0, 1, 0],
  [0, 0, 0],
  [0, 0, 0]
])

# up, right, down, left
filters = [conv_f, np.fliplr(conv_f.T), np.flipud(conv_f), conv_f.T]
conv = lambda inp, cf, fv: scipy.signal.convolve2d(inp, cf, 'same', fillvalue=fv)
dirs = np.array([conv(inp, flt, 9) > inp for flt in filters])
mask = np.all(dirs, axis=0)
sums = np.sum(inp[mask] + 1)
print(f"Day 9, puzzle 1: {sums}")

# day 9, puzzle 2

def flood(bitmask):
    masks = np.array([conv(bitmask, flt, 0) for flt in filters])
    mask = np.any(masks, axis=0)
    return mask | bitmask

def count_coord(inp, xy):
    global global_map
    step_mask = np.zeros_like(inp)
    step_mask[xy[0]][xy[1]] = 1
    last_count = None
    dirmask = (inp < 9)
    while True:
        step_mask = flood(step_mask) * dirmask
        count = np.sum(step_mask)
        if count == last_count:
            return count
        last_count = count

coords = np.argwhere(mask)
res = [count_coord(inp, cc) for cc in coords]
prod = np.prod(sorted(res)[-3:])
print(f"Day 9, puzzle 2: {prod}")

Day 9, puzzle 1: 562
Day 9, puzzle 2: 1076922


In [ ]:
inp = aoc_input(10)

# day 10, puzzle 1

openers = ('(', '[', '{', '<')
closers = (')', ']', '}', '>')
scores = (3, 57, 1197, 25137)

def puzzle_one(line):
    op_stack = []
    for ch in line:
        if ch in openers:
            idx = openers.index(ch)
            expect = closers[idx]
            op_stack.append(expect)
        else:
            expect = op_stack.pop()
            if ch != expect:
                idx = closers.index(ch)
                return scores[idx]
    return 0

sums = sum([puzzle_one(line) for line in inp])
print(f"Day 10, puzzle 1: {sums}")

# day 10, puzzle 2

def puzzle_two(line):
    op_stack = []
    for ch in line:
        if ch in openers:
            idx = openers.index(ch)
            expect = closers[idx]
            op_stack.append(expect)
        else:
            expect = op_stack.pop()
            if ch != expect:
                return 0
    total = 0
    for ch in op_stack[::-1]:
        total *= 5
        total += (closers.index(ch) + 1)
    return total

scores = [consume(line) for line in inp]
scores = sorted([score for score in scores if score > 0])
idx = len(scores) // 2
mid_score = scores[idx]
print(f"Day 10, puzzle 2: {mid_score}")


Day 10, puzzle 1: 266301
Day 10, puzzle 2: 3404870164


In [100]:
inp = aoc_input(11)
n_items = len(inp)
shape = (n_items, -1)
inp = str.join('', inp).encode()
inp = (np.frombuffer(inp, dtype=np.uint8).reshape(shape) - ord('0')).astype(np.int)

def get_tile(xy, shape):
    (x,y) = xy
    x = (max(0, x - 1), min(shape[0], x + 2))
    y = (max(0, y - 1), min(shape[1], y + 2))
    xy = (slice(*x), slice(*y))
    mask = np.ones((x[1] - x[0], y[1] - y[0]), dtype=np.int)
    return (xy, mask)

def step_dumbo(inp):
    inp += 1
    flashes = list(map(tuple, np.array(np.argwhere(inp == 10)).tolist()))
    seen = set()
    flash_count = 0
    while flashes:
        flash = flashes.pop()
        seen.add(flash)
        flash_count += 1
        (xy, mask) = get_tile(flash, inp.shape)
        inp[xy] += mask
        flashes = list(map(tuple, np.array(np.argwhere(inp == 10)).tolist())) + flashes
        flashes = list(set(flashes) - seen)
    
    # reset flash counter
    for flash in seen:
        inp[flash] = 0

    return flash_count

# day 11, puzzle 1

p1_inp = inp.copy()
sums = sum([step_dumbo(p1_inp) for idx in range(100)])
print(f"Day 11, puzzle 1: {sums}")

# day 11, puzzle 2

p2_inp = inp.copy()
step = 0
last_count = 0
while last_count != p2_inp.size:
    step += 1
    last_count = step_dumbo(p2_inp)

print(f"Day 11, puzzle 2: {step}")

Day 11, puzzle 1: 1642
Day 11, puzzle 2: 320
